In [1]:
from IPython.core.display import Markdown, display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))
import ipywidgets as widgets
from ipywidgets import interact, interact_manual

import traceback
import numpy as np
import scipy.stats as ss
import itertools
from os import path
import pickle
from sklearn.linear_model import LinearRegression, TheilSenRegressor, RANSACRegressor, HuberRegressor
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.preprocessing import PolynomialFeatures, StandardScaler
from sklearn.pipeline import make_pipeline

import matplotlib.pyplot as plt
plt.rcParams['figure.figsize'] = [13, 13]

import pandas as pd
pd.options.display.width = 500

slam_node_colors = {'gmapping': 'orange', 'slam_toolbox': 'blue', 'hector_slam': 'cyan'}


In [2]:
def load_cache(file_path):
    with open(path.expanduser(file_path), 'rb') as f:
        cache_dict = pickle.load(f)
        return cache_dict['df']

def printmd(*args):
    display(Markdown(' '.join(map(str, args))))

In [14]:
# load data from cache and rename columns
df_new_config = load_cache("/home/enrico/ds/performance_modelling/output/slam_new_config/run_data_per_waypoint_cache.pkl")
df_many_odoms = load_cache("/home/enrico/ds/performance_modelling/output/slam_many_odoms/run_data_per_waypoint_cache.pkl")
df = df_new_config.append(df_many_odoms, ignore_index=True, sort=False)
for i in df.columns: print(i);
    
df.rename(inplace=True, columns={
    'trajectory_length_trajectory_length': 'trajectory_length',
    'waypoint_relative_localization_error_translation_error_final': 'translation_error',
    'waypoint_absolute_localization_error_absolute_translation_error_start': 'absolute_translation_error',
    'waypoint_relative_localization_error_rotation_error_final': 'rotation_error',
    'geometric_similarity_sensor_mean_of_translation_eigenvalues_ratio_all': 'tgs_cont',
    'lidar_visibility_mean_visible_ranges_ratio': 'lvr_cont',
    'waypoint_start_time': 'start_time_cont',
    'trajectory_length_start_time': 'st',
    'trajectory_length_end_time': 'et',
})

# refactor and extract additional parameters and metrics
df['linear_update'] = df['linear_angular_update'].apply(lambda x: x[0])
df['beta'] = df['beta'].apply(lambda x: max(x))
df['norm_translation_error'] = df.translation_error / df.trajectory_length
df['run_id'] = df['run_folder'].apply(lambda x: path.basename(x))
df['waypoint_index'] = df.sort_values('start_time_cont').groupby(df.run_id).cumcount()
df['accumulated_trajectory_length_cont'] = df.sort_values('start_time_cont').groupby(df.run_id).trajectory_length.cumsum() - df.trajectory_length
df['session_id'] =  df['run_id'].apply(lambda x:  x.split('_')[1]+'_'+x.split('_')[2]+'_'+x.split('_')[3])
df['run_number'] =  df['run_id'].apply(lambda x:  int(x.split('_')[5]))
df['p'] = df.translation_error.apply(lambda x: x > 0.2)

# discretize continuous input metrics
df['tgs'] = pd.cut(df.tgs_cont, 5, labels=False)
df['lvr'] = pd.cut(df.lvr_cont, 10, labels=False)
df['start_time'] = pd.cut(df.start_time_cont, 10, labels=False)
df['accumulated_trajectory_length'] = pd.cut(df.accumulated_trajectory_length_cont, 10, labels=False)

# drop unnecessary datapoints
# df = df[(
#     ((df.slam_node == 'gmapping') | (df.slam_node == 'slam_toolbox')) & (df.linear_update == 0.5) #|
# #     (df.slam_node == 'hector_slam') & (df.linear_update == 0.4)
# )]
df = df[((df.slam_node == 'gmapping') | (df.slam_node == 'slam_toolbox')) & (df.linear_update == 0.5)]
df = df[df.trajectory_length > 1]
df = df[df.absolute_translation_error < 5]

# only keep necessary columns
robot_parameters = [
    'laser_scan_max_range',
    'laser_scan_fov_deg',
    'beta',
]
other_parameters = [
    'run_absolute_start_time',
    'session_id',
    'run_number',
    'run_id',
    'slam_node',
    'environment_name',
]
input_discretized_metrics = [
    'start_time',
    'accumulated_trajectory_length',
    'tgs',
    'lvr',
]
input_continuous_metrics = [
    'waypoint_index',
    'start_time_cont',
    'tgs_cont',
    'lvr_cont',
]
output_metrics = [
    'norm_translation_error',
    'absolute_translation_error',
    'p',
]
df = df[
    robot_parameters +
    other_parameters +
    input_discretized_metrics +
    input_continuous_metrics +
    output_metrics + 
    ['st', 'et']
]

mrs = sorted(list(df.laser_scan_max_range.unique()))
fovs = sorted(list(df.laser_scan_fov_deg.unique()))
betas = sorted(list(df.beta.unique()))
betas_fovs = list(itertools.product(betas, fovs))

printmd("## Parameters and Their Values")
for c in robot_parameters + input_discretized_metrics + ['slam_node', 'environment_name',]:
    printmd("{:>35}".format(c), sorted(list(df[c].unique())))

beta
ceres_loss_function
environment_name
fewer_nav_goals
geometric_similarity_end_time
geometric_similarity_mean_of_covariance_theta_theta
geometric_similarity_mean_of_covariance_x_x
geometric_similarity_mean_of_covariance_y_y
geometric_similarity_mean_of_translation_eigenvalues_ratio
geometric_similarity_mean_of_translation_eigenvalues_ratio_all
geometric_similarity_range_limit_end_time
geometric_similarity_range_limit_mean_of_covariance_theta_theta
geometric_similarity_range_limit_mean_of_covariance_x_x
geometric_similarity_range_limit_mean_of_covariance_y_y
geometric_similarity_range_limit_mean_of_translation_eigenvalues_ratio
geometric_similarity_range_limit_mean_of_translation_eigenvalues_ratio_all
geometric_similarity_range_limit_start_time
geometric_similarity_range_limit_version
geometric_similarity_sensor_end_time
geometric_similarity_sensor_mean_of_covariance_theta_theta
geometric_similarity_sensor_mean_of_covariance_x_x
geometric_similarity_sensor_mean_of_covariance_y_y
geo

## Parameters and Their Values

               laser_scan_max_range [3.5, 8.0, 15.0, 30.0]

                 laser_scan_fov_deg [90, 180, 270, 359]

                               beta [0.0, 0.5, 1.0, 1.5, 2.0]

...

      accumulated_trajectory_length [0, 1, 2, 3, 4, 5, 6, 7, 8, 9]

                                tgs [0, 1, 2, 3, 4]

                                lvr [0.0, 1.0, 2.0, 3.0, 4.0, 5.0, 6.0, 7.0, 8.0, 9.0]

                          slam_node ['gmapping', 'slam_toolbox']

                   environment_name ['7A-2', 'airlab', 'fr079', 'office_b']

In [9]:
print(len(df.run_id.unique()))

1747


In [52]:
model_params = robot_parameters+['tgs', 'slam_node']
for p in model_params:
    print(f"\n{p:>25}")
    for v in sorted(df[p].unique()):
        print(f"{v:>25}: {len(df[(df[p] == v)])}")


     laser_scan_max_range
                      3.5: 17934
                      8.0: 49290
                     15.0: 32758
                     30.0: 60241

       laser_scan_fov_deg
                       90: 20961
                      180: 51596
                      270: 31094
                      359: 56572

                     beta
                      0.0: 61024
                      0.5: 16403
                      1.0: 16419
                      1.5: 16134
                      2.0: 50243

                      tgs
                        0: 10188
                        1: 45398
                        2: 56405
                        3: 35647
                        4: 12585

                slam_node
                 gmapping: 79336
             slam_toolbox: 80887


In [38]:
print("Hours of trajectory execution:", (df.et - df.st).sum()/60/60)
print("Days of trajectory execution:", (df.et - df.st).sum()/60/60/24)

Hours of trajectory execution: 853.936148611111
Days of trajectory execution: 35.58067285879629
